#### Задача с качеством уроков

В нашей школе ученики и репетиторы занимаются в специальном онлайн-классе, в котором они могут общаться друг с другом, рисовать на доске, переписываться в чате, обмениваться файлами и решать различные задачки. Информацию о каждом уроке мы старательно записываем в базу данных в таблицу lessons. На каждом таком уроке присутствует один репетитор и один ученик (существуют еще тренировочные уроки, у которых единственный участник – репетитор, но для нашей задачи они неинтересны). В конце урока, когда учитель и ученик покидают класс, они могут оценить (а могут и пропустить этот этап) качество пройденного урока по пятибальной шкале. Все оценки мы записываем в таблицу quality, чтобы мы могли узнавать, хорошо ли работает наш сайт.

В этой задаче тебе нужно будет в каждый день найти репетитора, уроки которого получили самую низкую оценку качества, и вывести эту оценку.

В архиве содержится четыре файла с выгрузкой строк из базы.

lessons.txt содержит следующие поля:
id – уникальный идентификатор урока.
event_id – идентификатор, связывающий уроки с файлом participants. У нескольких уроков может быть один и тот же event_id. Подробнее про это будет позже
subject – предмет урока, просто строка
scheduled_time – время начала урока, в формате ГГГГ-ММ-ДД чч:мм:сс (иногда есть еще миллисекунды). Время указано в UTC.

quality.txt содержит следующие поля:
lesson_id – идентификатор урока, указывает на запись в таблице lessons.txt. У нескольких строчек из этого файла может быть один и тот же lesson_id, потому что оценок за урок может быть несколько (например, оба участника, ученик и репетитор могут оценить качество урока)
tech_quality – собственно оценка качества урока. Это число от 1 до 5. Иногда его может не быть, если пользователь не выставил оценку.

users.txt хранит информацию о пользователях и содержит всего два поля:
id – уникальный идентификатор пользователя
role – указывает является ли пользователь учеником (pupil) или учителем (tutor)

Файл participants.txt позволяет связать урок с его участниками. Он содержит следующие поля:
user_id – идентификатор пользователя (указывает на запись в файле users.txt)
event_id – идентификатор, связывающий урок с участником. То есть, чтобы понять, какие пользователи были на уроке Х, нужно найти в файле participants.txt строки, у которых event_id совпадает с event_id урока Х.

Для решения задачки нужно сделать следующее:
1. Найти все уроки по физике (subject=phys).
2. В каждый день (начало и конец дня считается по московскому времени, то есть UTC+3:00) для каждого репетитора посчитать среднюю арифметическую оценку за его уроки (учитывать только уроки из п. 1). То есть, если учитель проводил в этот день три урока по физике, один из них он оценил на 3, а ученик оценил его на 4, второму уроку оценку поставил только ученик, и эта оценка 5, а третий урок вообще никто не оценил, то средняя арифметическая оценка учителя за уроки = (3 + 4 + 5) / 3 = 4.
3. Найти учителя, который в этот день имеет самую низкую среднюю арифметическую оценку за уроки (среди всех учителей, проводивших уроки по физике в этот день).
4. Вывести его в формате "<день> <id учителя> <средняя арифметическая оценка>". Оценку можно округлить с точностью до двух знаков после запятой. То есть как-то так:
2020-01-11 73c9af08-8581-430c-a590-9888ab36deb3 3.67
2020-01-12 909c2c8e-c054-4e9f-a51a-50bf5660f364 3.25
...
4. Учитывать нужно только тех учителей, за уроки по физике которых в этот день стоит хотя бы одна оценка. Если у нескольких учителей одна и та же самая оценка, можно вывести любого.
5. Если есть необходимость, предварительно можно предобработать данные (сконвертировав их в csv или загрузив в базу данных) и работать уже с ними.

In [1]:
import pandas as pd

In [2]:
lessons = pd.read_csv('lessons.txt', sep='|', index_col=False)
lessons = lessons.dropna()
lessons.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378 entries, 1 to 378
Data columns (total 4 columns):
                  id                      378 non-null object
 event_id                                 378 non-null float64
 subject                                  378 non-null object
       scheduled_time                     378 non-null object
dtypes: float64(1), object(3)
memory usage: 14.8+ KB


In [3]:
lessons.head()

,id,event_id,subject,scheduled_time
1,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114.0,phys,2020-01-19 12:00:00
2,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,51568.0,it,2020-01-19 13:00:00
3,62e1a078-33de-47c1-99d2-845b1daca56f,52790.0,hist,2020-01-19 13:00:00
4,00fc6685-f53a-49bb-b960-5e0042fd3852,51341.0,phys,2020-01-17 12:00:00
5,4cadf623-82e6-422f-a342-acf978302fb2,55048.0,phys,2020-01-19 14:00:00


In [4]:
lessons.set_axis(['lesson_id', 'event_id', 'subject', 'scheduled_time'], axis='columns', inplace=True)
lessons.head()

,lesson_id,event_id,subject,scheduled_time
1,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114.0,phys,2020-01-19 12:00:00
2,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,51568.0,it,2020-01-19 13:00:00
3,62e1a078-33de-47c1-99d2-845b1daca56f,52790.0,hist,2020-01-19 13:00:00
4,00fc6685-f53a-49bb-b960-5e0042fd3852,51341.0,phys,2020-01-17 12:00:00
5,4cadf623-82e6-422f-a342-acf978302fb2,55048.0,phys,2020-01-19 14:00:00


In [5]:
lessons['event_id'] = lessons['event_id'].astype('int')
lessons['scheduled_time'] = pd.to_datetime(lessons['scheduled_time'], format='%Y-%m-%d %H:%M:%S')
lessons['moscow_time'] = lessons['scheduled_time'] + pd.Timedelta(hours=3)
lessons.head()

,lesson_id,event_id,subject,scheduled_time,moscow_time
1,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114,phys,2020-01-19 12:00:00,2020-01-19 15:00:00
2,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,51568,it,2020-01-19 13:00:00,2020-01-19 16:00:00
3,62e1a078-33de-47c1-99d2-845b1daca56f,52790,hist,2020-01-19 13:00:00,2020-01-19 16:00:00
4,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00,2020-01-17 15:00:00
5,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 14:00:00,2020-01-19 17:00:00


In [6]:
lessons.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378 entries, 1 to 378
Data columns (total 5 columns):
lesson_id         378 non-null object
event_id          378 non-null int64
subject           378 non-null object
scheduled_time    378 non-null datetime64[ns]
moscow_time       378 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 17.7+ KB


In [7]:
lessons['subject'].value_counts()

 phys        184
 bio          78
 hist         60
 it           56
Name: subject, dtype: int64

In [8]:
phys_lessons = lessons[lessons['subject'] == ' phys    '].reset_index(drop=True)
phys_lessons.head()

,lesson_id,event_id,subject,scheduled_time,moscow_time
0,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114,phys,2020-01-19 12:00:00,2020-01-19 15:00:00
1,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00,2020-01-17 15:00:00
2,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 14:00:00,2020-01-19 17:00:00
3,e597a79a-3f68-4d15-bc77-c7000bcf8e52,53026,phys,2020-01-19 15:00:00,2020-01-19 18:00:00
4,ea6f0bf3-bcfc-4555-9b16-cc2cfe722e17,51645,phys,2020-01-19 14:00:00,2020-01-19 17:00:00


In [9]:
phys_lessons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 5 columns):
lesson_id         184 non-null object
event_id          184 non-null int64
subject           184 non-null object
scheduled_time    184 non-null datetime64[ns]
moscow_time       184 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 7.3+ KB


In [10]:
quality = pd.read_csv('quality.txt', sep='|')
quality = quality.dropna()
quality.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365 entries, 1 to 365
Data columns (total 2 columns):
              lesson_id                   365 non-null object
 tech_quality                             365 non-null object
dtypes: object(2)
memory usage: 8.6+ KB


In [11]:
quality.set_axis(['lesson_id', 'tech_quality'], axis='columns', inplace=True)
quality.head()

,lesson_id,tech_quality
1,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,5
2,62e1a078-33de-47c1-99d2-845b1daca56f,5
3,62e1a078-33de-47c1-99d2-845b1daca56f,5
4,00fc6685-f53a-49bb-b960-5e0042fd3852,5
5,00fc6685-f53a-49bb-b960-5e0042fd3852,5


In [12]:
quality['tech_quality'].value_counts()

            5    307
            4     24
                  16
            3      9
            2      6
            1      3
Name: tech_quality, dtype: int64

In [13]:
quality['tech_quality'] = pd.to_numeric(quality['tech_quality'], errors='coerce')

In [14]:
quality['tech_quality'].value_counts()

5.0    307
4.0     24
3.0      9
2.0      6
1.0      3
Name: tech_quality, dtype: int64

In [15]:
quality_table = quality.pivot_table(index='lesson_id', values='tech_quality')

In [16]:
lessons_quality = phys_lessons.merge(quality_table, how='inner', on='lesson_id')
lessons_quality.head()

,lesson_id,event_id,subject,scheduled_time,moscow_time,tech_quality
0,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00,2020-01-17 15:00:00,5.0
1,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 14:00:00,2020-01-19 17:00:00,5.0
2,e597a79a-3f68-4d15-bc77-c7000bcf8e52,53026,phys,2020-01-19 15:00:00,2020-01-19 18:00:00,5.0
3,ea6f0bf3-bcfc-4555-9b16-cc2cfe722e17,51645,phys,2020-01-19 14:00:00,2020-01-19 17:00:00,5.0
4,7d56b43a-d137-446d-8570-8d7d5ccbaee9,51153,phys,2020-01-20 17:00:00,2020-01-20 20:00:00,5.0


In [17]:
participants = pd.read_csv('participants.txt', sep='|')
participants = participants.dropna()
participants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 743 entries, 1 to 743
Data columns (total 2 columns):
 event_id                                 743 non-null object
               user_id                    743 non-null object
dtypes: object(2)
memory usage: 17.4+ KB


In [18]:
participants.set_axis(['event_id', 'user_id'], axis='columns', inplace=True)
participants.head()

,event_id,user_id
1,38114,e28351f5-4ccb-4549-8647-d43f2b15e7b8
2,38114,4df2832a-1d63-4453-9659-43993fc35996
3,51568,bb1c0bc8-1212-452b-97a0-439d4a2169e2
4,51568,63441abe-c4da-4275-ba26-66f7dbd65dde
5,52790,a1323b68-c82c-429a-8e2c-65597e648c1a


In [19]:
participants['event_id'] = participants['event_id'].astype('int')

In [20]:
participants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 743 entries, 1 to 743
Data columns (total 2 columns):
event_id    743 non-null int64
user_id     743 non-null object
dtypes: int64(1), object(1)
memory usage: 17.4+ KB


In [21]:
users = pd.read_csv('users.txt', sep='|')
users = users.dropna()
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 743 entries, 1 to 743
Data columns (total 2 columns):
                  id                      743 non-null object
 role                                     743 non-null object
dtypes: object(2)
memory usage: 17.4+ KB


In [22]:
users.set_axis(['user_id', 'role'], axis='columns', inplace=True)
users.head()

,user_id,role
1,e28351f5-4ccb-4549-8647-d43f2b15e7b8,pupil
2,4df2832a-1d63-4453-9659-43993fc35996,tutor
3,bb1c0bc8-1212-452b-97a0-439d4a2169e2,pupil
4,63441abe-c4da-4275-ba26-66f7dbd65dde,tutor
5,a1323b68-c82c-429a-8e2c-65597e648c1a,tutor


In [23]:
participants_users = participants.merge(users, on='user_id', how='left', left_index=True, right_index=True)

In [24]:
participants_users.head()

,event_id,user_id,role
1,38114,e28351f5-4ccb-4549-8647-d43f2b15e7b8,pupil
2,38114,4df2832a-1d63-4453-9659-43993fc35996,tutor
3,51568,bb1c0bc8-1212-452b-97a0-439d4a2169e2,pupil
4,51568,63441abe-c4da-4275-ba26-66f7dbd65dde,tutor
5,52790,a1323b68-c82c-429a-8e2c-65597e648c1a,tutor


In [25]:
participants_tutors = participants_users[participants_users['role'] == ' tutor']
participants_tutors.head()

,event_id,user_id,role
2,38114,4df2832a-1d63-4453-9659-43993fc35996,tutor
4,51568,63441abe-c4da-4275-ba26-66f7dbd65dde,tutor
5,52790,a1323b68-c82c-429a-8e2c-65597e648c1a,tutor
8,51341,30a19496-bdaf-461c-abbc-2709ae520201,tutor
9,55048,43efce48-94b2-4412-857f-223d45969008,tutor


In [26]:
sub_total_data = lessons_quality.merge(participants_tutors, how='inner', on='event_id')
sub_total_data.head()

,lesson_id,event_id,subject,scheduled_time,moscow_time,tech_quality,user_id,role
0,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00,2020-01-17 15:00:00,5.0,30a19496-bdaf-461c-abbc-2709ae520201,tutor
1,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 14:00:00,2020-01-19 17:00:00,5.0,43efce48-94b2-4412-857f-223d45969008,tutor
2,e597a79a-3f68-4d15-bc77-c7000bcf8e52,53026,phys,2020-01-19 15:00:00,2020-01-19 18:00:00,5.0,43efce48-94b2-4412-857f-223d45969008,tutor
3,ea6f0bf3-bcfc-4555-9b16-cc2cfe722e17,51645,phys,2020-01-19 14:00:00,2020-01-19 17:00:00,5.0,8fe03f08-8581-430c-a590-9888ab36deb3,tutor
4,ea6f0bf3-bcfc-4555-9b16-cc2cfe722e17,51645,phys,2020-01-19 14:00:00,2020-01-19 17:00:00,5.0,8fe03f08-8581-430c-a590-9888ab36deb3,tutor


In [27]:
sub_total_data['moscow_time_date'] = sub_total_data['moscow_time'].dt.floor('1D')
sub_total_data.head()

,lesson_id,event_id,subject,scheduled_time,moscow_time,tech_quality,user_id,role,moscow_time_date
0,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00,2020-01-17 15:00:00,5.0,30a19496-bdaf-461c-abbc-2709ae520201,tutor,2020-01-17
1,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 14:00:00,2020-01-19 17:00:00,5.0,43efce48-94b2-4412-857f-223d45969008,tutor,2020-01-19
2,e597a79a-3f68-4d15-bc77-c7000bcf8e52,53026,phys,2020-01-19 15:00:00,2020-01-19 18:00:00,5.0,43efce48-94b2-4412-857f-223d45969008,tutor,2020-01-19
3,ea6f0bf3-bcfc-4555-9b16-cc2cfe722e17,51645,phys,2020-01-19 14:00:00,2020-01-19 17:00:00,5.0,8fe03f08-8581-430c-a590-9888ab36deb3,tutor,2020-01-19
4,ea6f0bf3-bcfc-4555-9b16-cc2cfe722e17,51645,phys,2020-01-19 14:00:00,2020-01-19 17:00:00,5.0,8fe03f08-8581-430c-a590-9888ab36deb3,tutor,2020-01-19


In [28]:
sub_total_pivot = sub_total_data.pivot_table(index=['moscow_time_date', 'user_id'], values='tech_quality')

In [29]:
sub_total_pivot

tech_quality
moscow_time_date user_id                                            
2020-01-11        20ee67a0-1c94-4828-9a09-49c16b65f4b5      5.000000
                  2fa2ab62-f1b0-4036-872f-bcfd9a8686ff      5.000000
                  30a19496-bdaf-461c-abbc-2709ae520201      5.000000
                  43efce48-94b2-4412-857f-223d45969008      5.000000
                  603b8641-c6f6-4d89-ac89-88e50d45aa0d      4.900000
                  8fe03f08-8581-430c-a590-9888ab36deb3      4.600000
                  b37ccae8-fc31-4ad8-8f55-ca855b23fbf6      5.000000
                  c0c073c8-cb9c-4ebc-ac13-472c06029591      5.000000
                  f2f25148-97cb-422d-a5b5-b0dabbca0523      3.000000
                  f6263582-489a-4abc-b19d-a2381b026e28      5.000000
2020-01-12        0d688d4b-b5b7-4519-ab32-785138b3a330      5.000000
                  2fa2ab62-f1b0-4036-872f-bcfd9a8686ff      5.000000
                  38e5889b-ecdb-428b-9e9d-fb786ad35702      5.000000
                  43efce48-94b2-4412-857f-223d45969008      5.000000
                  4df2832a-1d63-4453-9659-43993fc35996      5.000000
                  5b00e607-13c0-40e4-aacf-22142d339074      5.000000
                  696c838e-c054-4e9f-a51a-50bf5660f364      4.900000
                  8fe03f08-8581-430c-a590-9888ab36deb3      5.000000
                  a0a1ca0f-0de5-4f1c-a729-d7c4792457aa      5.000000
                  ac9d73e0-1b42-4031-8edd-2dce14fa3305      5.000000
                  b1fe2c06-d284-4698-9f07-495c8f991700      5.000000
                  b37ccae8-fc31-4ad8-8f55-ca855b23fbf6      5.000000
                  c8b6b1e9-c1a5-441c-8649-44d80b8a365b      5.000000
2020-01-13        2fa2ab62-f1b0-4036-872f-bcfd9a8686ff      5.000000
                  30a19496-bdaf-461c-abbc-2709ae520201      5.000000
                  696c838e-c054-4e9f-a51a-50bf5660f364      5.000000
                  b37ccae8-fc31-4ad8-8f55-ca855b23fbf6      5.000000
                  be676776-8366-4c71-8a35-d58014806eb5      5.000000
2020-01-14        2fa2ab62-f1b0-4036-872f-bcfd9a8686ff      5.000000
                  43efce48-94b2-4412-857f-223d45969008      5.000000
...                                                              ...
2020-01-18        603b8641-c6f6-4d89-ac89-88e50d45aa0d      4.600000
                  8fe03f08-8581-430c-a590-9888ab36deb3      5.000000
                  96d9da9a-e1e0-459e-ba8f-724d51d5b3cf      5.000000
                  9d9029a2-c81a-49f7-84ee-7696eecd2f99      2.000000
                  abac0e59-f8b1-4f91-a6e9-ec062719cabd      5.000000
                  b37ccae8-fc31-4ad8-8f55-ca855b23fbf6      5.000000
                  c0c073c8-cb9c-4ebc-ac13-472c06029591      5.000000
                  f2f25148-97cb-422d-a5b5-b0dabbca0523      5.000000
                  f6263582-489a-4abc-b19d-a2381b026e28      5.000000
2020-01-19        0d688d4b-b5b7-4519-ab32-785138b3a330      5.000000
                  38e5889b-ecdb-428b-9e9d-fb786ad35702      5.000000
                  3c9ccac9-6180-4e74-993e-c83f15a9b6c8      5.000000
                  43efce48-94b2-4412-857f-223d45969008      5.000000
                  4df2832a-1d63-4453-9659-43993fc35996      5.000000
                  5b00e607-13c0-40e4-aacf-22142d339074      5.000000
                  696c838e-c054-4e9f-a51a-50bf5660f364      4.833333
                  8fe03f08-8581-430c-a590-9888ab36deb3      5.000000
                  9c2ccb50-b72b-49c1-95b3-3e9cee9d512e      5.000000
                  a0a1ca0f-0de5-4f1c-a729-d7c4792457aa      5.000000
                  ac9d73e0-1b42-4031-8edd-2dce14fa3305      5.000000
                  b1fe2c06-d284-4698-9f07-495c8f991700      4.750000
                  b323a494-40c0-4522-8d63-4d4d591a1def      4.500000
                  be676776-8366-4c71-8a35-d58014806eb5      4.500000
                  c8b6b1e9-c1a5-441c-8649-44d80b8a365b      5.000000
2020-01-20        2fa2ab62-f1b0-4036-872f-bcfd9a8686ff      5.000000
                  30a19496-bdaf-461c-abbc-2709ae520201

In [30]:
sub_total = sub_total_pivot.groupby('moscow_time_date').min()
sub_total

,tech_quality
moscow_time_date,
2020-01-11,3.0
2020-01-12,4.9
2020-01-13,5.0
2020-01-14,4.0
2020-01-15,5.0
2020-01-16,4.0
2020-01-17,4.5
2020-01-18,2.0
2020-01-19,4.5


In [44]:
min_quality_list = sub_total['tech_quality'].tolist()
print(min_quality_list)

[3.0, 4.9, 5.0, 4.0, 5.0, 4.0, 4.5, 2.0, 4.5, 4.5]


In [57]:
df = sub_total_pivot['tech_quality'].reset_index()
df

,moscow_time_date,user_id,tech_quality
0,2020-01-11,20ee67a0-1c94-4828-9a09-49c16b65f4b5,5.000000
1,2020-01-11,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,5.000000
2,2020-01-11,30a19496-bdaf-461c-abbc-2709ae520201,5.000000
3,2020-01-11,43efce48-94b2-4412-857f-223d45969008,5.000000
4,2020-01-11,603b8641-c6f6-4d89-ac89-88e50d45aa0d,4.900000
5,2020-01-11,8fe03f08-8581-430c-a590-9888ab36deb3,4.600000
6,2020-01-11,b37ccae8-fc31-4ad8-8f55-ca855b23fbf6,5.000000
7,2020-01-11,c0c073c8-cb9c-4ebc-ac13-472c06029591,5.000000
8,2020-01-11,f2f25148-97cb-422d-a5b5-b0dabbca0523,3.000000
9,2020-01-11,f6263582-489a-4abc-b19d-a2381b026e28,5.000000


In [62]:
df11 = df.loc[df['moscow_time_date'] == '2020-01-11']
df11

,moscow_time_date,user_id,tech_quality
0,2020-01-11,20ee67a0-1c94-4828-9a09-49c16b65f4b5,5.0
1,2020-01-11,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,5.0
2,2020-01-11,30a19496-bdaf-461c-abbc-2709ae520201,5.0
3,2020-01-11,43efce48-94b2-4412-857f-223d45969008,5.0
4,2020-01-11,603b8641-c6f6-4d89-ac89-88e50d45aa0d,4.9
5,2020-01-11,8fe03f08-8581-430c-a590-9888ab36deb3,4.6
6,2020-01-11,b37ccae8-fc31-4ad8-8f55-ca855b23fbf6,5.0
7,2020-01-11,c0c073c8-cb9c-4ebc-ac13-472c06029591,5.0
8,2020-01-11,f2f25148-97cb-422d-a5b5-b0dabbca0523,3.0
9,2020-01-11,f6263582-489a-4abc-b19d-a2381b026e28,5.0


In [68]:
df111 = df11.loc[df11['tech_quality'] == 3.0].reset_index(drop=True)
df111

,moscow_time_date,user_id,tech_quality
0,2020-01-11,f2f25148-97cb-422d-a5b5-b0dabbca0523,3.0
